In [27]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import cv2

In [5]:
def extract_frames(video_path,output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    # Read and save frames
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Save frame
        frame_name = f"frame_{frame_count:04d}.jpg"
        frame_path = os.path.join(output_folder, frame_name)
        cv2.imwrite(frame_path, frame)

        frame_count += 1

    cap.release()
    print(f"{frame_count} frames extracted and saved to {output_folder}")


In [6]:
def extract_frames_from_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(input_folder):
        if filename.endswith((".mp4", ".avi", ".mov")):
            video_path = os.path.join(input_folder, filename)
            extract_frames(video_path, output_folder)

In [7]:
extract_frames_from_folder("../Posture/Correct","../Correct_Posture")

1090 frames extracted and saved to ../Correct_Posture


In [8]:
extract_frames_from_folder("../Posture/Incorrect","../Inorrect_Posture")

703 frames extracted and saved to ../Inorrect_Posture
1025 frames extracted and saved to ../Inorrect_Posture


In [28]:
train_dir = '../Posture_Data'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2    
)



In [29]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    subset='training'    
)

Found 1692 images belonging to 2 classes.


In [30]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    subset='validation'    
)

Found 423 images belonging to 2 classes.


In [38]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax')
])

In [39]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [40]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 20,  
    epochs=15,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 20   
)


Epoch 1/15


ValueError: in user code:

    File "c:\Users\anish\miniconda3\envs\ml\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\anish\miniconda3\envs\ml\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\anish\miniconda3\envs\ml\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\anish\miniconda3\envs\ml\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\anish\miniconda3\envs\ml\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\anish\miniconda3\envs\ml\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_va

In [34]:
model.save("Posture_Model.h5")